In [1]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
import numpy as np

In [2]:
import pandas as pd

# Read the CSV file
data = pd.read_csv("/kaggle/input/english-nagamese-raw-data/raw_data.csv")

# Verify the data
print(data.head())


                                             English  \
0  Paul, Silvanus, and Timothy to the church of t...   
1  We always give thanks to God for all of you as...   
2  We remember before our God and Father your wor...   
3  Brothers loved by God, we know he has chosen you,   
4  because our gospel came to you not in word onl...   

                                            Nagamese  
0  Paul aru Silvanus aru Timothy pora Isor aru Pr...  
1  Amikhan hodai apnikhan nimite Isor ke dhanyaba...  
2  Amikhan pora apni khan laga biswas laga kaam, ...  
3  Isor pora morom kora bhai khan, amikhan jane T...  
4  kilemane Isor laga kotha apni khan logote khal...  


In [3]:
# data=data[:100]

In [4]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load model and tokenizer
model_name = "facebook/m2m100_418M"
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)



config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [5]:
# # Prepare dataset
# source_texts = list(data['English'])
# target_texts = list(data['Nagamese'])

# # Tokenize the data
# inputs = tokenizer(source_texts, return_tensors="pt", max_length=128, truncation=True, padding="max_length")
# labels = tokenizer(target_texts, return_tensors="pt", max_length=128, truncation=True, padding="max_length")["input_ids"]

# # Adjust labels for loss calculation
# labels[labels == tokenizer.pad_token_id] = -100

# # Create DataLoader
# from torch.utils.data import DataLoader, TensorDataset

# dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"], labels)
# dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# # Define optimizer
# optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# # Fine-tuning loop
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [6]:
source_texts = list(data["English"])  
target_texts = list(data["Nagamese"]) 



# Split the data into training and validation sets
train_source_texts, val_source_texts, train_target_texts, val_target_texts = train_test_split(
    source_texts, target_texts, test_size=0.20, random_state=50
)


print(type(train_source_texts))
print(type(train_target_texts))
print(type(val_source_texts))
print(type(val_target_texts))


<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [7]:

# Tokenize the training and validation data
train_inputs = tokenizer(train_source_texts, return_tensors="pt", max_length=128, truncation=True, padding="max_length")
train_labels = tokenizer(train_target_texts, return_tensors="pt", max_length=128, truncation=True, padding="max_length")["input_ids"]

val_inputs = tokenizer(val_source_texts, return_tensors="pt", max_length=128, truncation=True, padding="max_length")
val_labels = tokenizer(val_target_texts, return_tensors="pt", max_length=128, truncation=True, padding="max_length")["input_ids"]

# Adjust labels for loss calculation
train_labels[train_labels == tokenizer.pad_token_id] = -100
val_labels[val_labels == tokenizer.pad_token_id] = -100

# Create DataLoader for training and validation
train_dataset = TensorDataset(train_inputs["input_ids"], train_inputs["attention_mask"], train_labels)
val_dataset = TensorDataset(val_inputs["input_ids"], val_inputs["attention_mask"], val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Early stopping setup
patience = 100  # Number of batches to wait for improvement before stopping
best_val_loss = float("inf")
batches_without_improvement = 0

In [8]:
# Fine-tuning setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 6
validation_interval = 80  # Number of batches after which validation is performed

batch_counter = 0

for epoch in range(epochs):
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        batch_counter += 1

        # Perform validation after every 'validation_interval' batches
        if batch_counter % validation_interval == 0:
            model.eval()
            total_val_loss = 0
            with torch.no_grad():
                for val_batch in val_dataloader:
                    input_ids, attention_mask, labels = [x.to(device) for x in val_batch]
                    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    val_loss = outputs.loss
                    total_val_loss += val_loss.item()

            avg_val_loss = total_val_loss / len(val_dataloader)
            print(f"Batch {batch_counter}, Training Loss: {loss.item()}, Validation Loss: {avg_val_loss}")

            # Early stopping condition
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                batches_without_improvement = 0
                # Save model checkpoint
                model.save_pretrained("./m2m100-finetuned-english-nagamese")
                tokenizer.save_pretrained("./m2m100-finetuned-english-nagamese")
            else:
                batches_without_improvement += 1

            if batches_without_improvement >= patience:
                print(f"Early stopping triggered after {batch_counter} batches due to no improvement.")
                break

    if batches_without_improvement >= patience:
        break

Batch 80, Training Loss: 2.8972980976104736, Validation Loss: 2.651746246814728


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Batch 160, Training Loss: 2.074230432510376, Validation Loss: 2.0240095973014833
Batch 240, Training Loss: 1.7577309608459473, Validation Loss: 1.7545105969905854
Batch 320, Training Loss: 1.877717137336731, Validation Loss: 1.6209443151950835
Batch 400, Training Loss: 1.5675188302993774, Validation Loss: 1.5596867573261262
Batch 480, Training Loss: 1.3761250972747803, Validation Loss: 1.4978657960891724
Batch 560, Training Loss: 1.1408917903900146, Validation Loss: 1.4746029913425445
Batch 640, Training Loss: 1.353389024734497, Validation Loss: 1.4356591057777406
Batch 720, Training Loss: 1.1522822380065918, Validation Loss: 1.4269418466091155
Batch 800, Training Loss: 1.0307962894439697, Validation Loss: 1.3865585768222808
Batch 880, Training Loss: 0.799170196056366, Validation Loss: 1.4065670573711395
Batch 960, Training Loss: 0.8323827385902405, Validation Loss: 1.4173873901367187
Batch 1040, Training Loss: 1.0309982299804688, Validation Loss: 1.4108629262447356
Batch 1120, Trainin

In [9]:
# Final save of the fine-tuned model if early stopping was not triggered
model.save_pretrained("./m2m100-finetuned-english-nagamese-final")
tokenizer.save_pretrained("./m2m100-finetuned-english-nagamese-final")

('./m2m100-finetuned-english-nagamese-final/tokenizer_config.json',
 './m2m100-finetuned-english-nagamese-final/special_tokens_map.json',
 'm2m100-finetuned-english-nagamese-final/vocab.json',
 'm2m100-finetuned-english-nagamese-final/sentencepiece.bpe.model',
 './m2m100-finetuned-english-nagamese-final/added_tokens.json')

In [10]:
# epochs = 3
# for epoch in range(epochs):
#     model.train()
#     i = 0
#     for batch in dataloader:
#         input_ids, attention_mask, labels = [x.to(device) for x in batch]
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()
#         print("iterating ", i)
#         i = i+1
#     print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# # Save the fine-tuned model
# model.save_pretrained("./m2m100-finetuned-english-nagamese")
# tokenizer.save_pretrained("./m2m100-finetuned-english-nagamese")

In [11]:
!pip install transformers sacrebleu
import os
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import sacrebleu
import pandas as pd
from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.0 MB/s eta 0:00:00


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [13]:
model = model.to(device)

In [14]:
source_texts = val_source_texts
reference_texts = val_target_texts

In [15]:
tokenizer.lang_code_to_id["nag"] = tokenizer.lang_code_to_id.get("eng") 

In [16]:
# Translate source texts using the model
model.eval()
machine_translations = []

for text in source_texts:
    inputs = tokenizer(text, return_tensors="pt", max_length=64, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["nag"])
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    machine_translations.append(translated_text)

In [17]:
import sacrebleu

# Ensure reference_texts is in the required format
wrapped_reference_texts = [reference_texts] 

# Compute BLEU score
bleu = sacrebleu.corpus_bleu(machine_translations, wrapped_reference_texts)
print("BLEU Score:", bleu.score)

BLEU Score: 21.512355890568216
